In [ ]:
import os 
import json
import re

In [ ]:
exp_name = "MiniCPM"
round = 1
episode_id = 1
flag = 'all'
task_type = 'all'

In [ ]:
if flag == 'train':
    ran = range(1,91)
elif flag == 'test':
    ran = range(91,101)
else:
    ran = range(1,101)

In [ ]:
task_list = {}
task_list['typical'] = [2, 7, 9, 19, 24, 27, 28, 31, 46, 47, 49, 60, 61, 70, 91, 92]
task_list['long-horizon'] = [1, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 25, 26, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 93, 94, 95, 96, 97, 98, 99, 100]
task_list['open-ended'] = [20, 22, 44, 69, 76, 77, 96, 97, 98]
task_list['logical'] = [1, 2, 3, 4, 12, 14, 15, 17, 18, 19, 20, 23, 26, 27, 34, 37, 38, 40, 42, 43, 44, 45, 52, 55, 56, 57, 59, 62, 64, 68, 69, 71, 72, 73, 75, 76, 77, 80, 81, 83, 84, 85, 86, 87, 88, 94, 96, 98, 100]
task_list['human style'] = [1, 2, 3, 4, 5, 6, 8, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 26, 27, 29, 33, 34, 36, 37, 38, 40, 41, 42, 43, 44, 45, 49, 51, 52, 53, 54, 55, 56, 57, 59, 61, 62, 63, 64, 66, 68, 69, 71, 72, 73, 75, 76, 77, 79, 80, 82, 83, 84, 85, 86, 87, 88, 91, 94, 98, 96, 97, 100]
if task_type != 'all':
    ran = task_list[task_type]

In [ ]:
def parse_execution_history(history_string):
    history_string = history_string.replace("Historical Execution: ", "")
    historty_splits = re.split(r'(\(fail\)|\(success\))\s', history_string)

    parsed_steps = []

    for i in range(0, len(historty_splits)-1, 2):
        result = 'success' if 'success' in historty_splits[i+1] else 'fail'
        action_parts = re.sub(r'\(.*?\)', '', historty_splits[i], 1).strip().lstrip('[').rstrip(']').split(',')
        
        parsed_step = [
            action_parts[0].strip(),
            action_parts[1].strip().lower() if len(action_parts) > 1 else "None",
            action_parts[2].strip().lower() if len(action_parts) > 2 else "None",
            result
        ]
        parsed_steps.append(parsed_step)

    return parsed_steps

In [ ]:
def eval_list_gen(exp_name, round):

    exp_path = os.path.join("exp", exp_name)

    eval_list = []

    for index in os.listdir(exp_path):
        conv_path = os.path.join(exp_path, index, f"{round}/conversation.json")
        if not os.path.exists(conv_path):
            continue

        with open(conv_path, 'r') as file:
            conv = json.load(file)  
        exe_dict = {}
        exe_dict["index"] = int(index) 
        exe_dict["step_len"] = len(conv)

        history = conv[-1].split('\n')[3]
        exe_dict["history"] = parse_execution_history(history)
        eval_list.append(exe_dict) 

    eval_list = sorted(eval_list, key=lambda x: x["index"])
    print(len(eval_list))
    return eval_list

In [ ]:
eval_list = eval_list_gen(exp_name,round)
eval_list[0]

In [ ]:
def eval_success_list_gen(exp_name,round):

    exp_path = os.path.join("exp", exp_name)

    eval_success_list = []

    for index in os.listdir(exp_path):
        conv_path = os.path.join(exp_path, index, f"{round}/conversation.json")
        if not os.path.exists(conv_path):
            continue

        with open(conv_path, 'r') as file:
            conv = json.load(file)
        if 'End' in conv[-1]:   
            exe_dict = {}
            exe_dict["index"] = int(index) 
            exe_dict["step_len"] = len(conv)

            history = conv[-1].split('\n')[3]
            exe_dict["history"] = parse_execution_history(history)
            exe_dict["history"].append(["End", "None", "None", "success"])
            eval_success_list.append(exe_dict)

    eval_success_list = sorted(eval_success_list, key=lambda x: x["index"])
    print(len(eval_success_list))

    return eval_success_list

def load_name_mapping():
        with open('hab-mobile-manipulation/name_dict.txt', 'r') as file:
            content = file.read()
        lines = content.split('\n')
        print(lines[-1])
        
        name_dict = {}
        for i in range(0, len(lines), 3):
            value = lines[i].strip().strip(':')
            key = lines[i + 1].strip()
            keys = key.split('/')
            for i in keys:
                name_dict[i] = value

        return name_dict
    # mapping item name
mapping_dict = load_name_mapping()

def match_action(action, keypoint, mapping_dict):
        #print(keypoint)
        if action[3] == 'fail':
               return False

        key_action = [element.strip() for element in keypoint.strip('[').strip(']').split(',')]
        if len(key_action) == 2:
            key_action.append("None")
        if action[0] == key_action[0]:
                if key_action[0] == "End":
                    return True
                obj1 = mapping_dict[action[1]]
                obj2 = mapping_dict[key_action[1]]
                if obj1 == obj2:
                       obj11 = "None" if action[2] == "None" else mapping_dict[action[2]]
                       obj22 = "None" if key_action[2] == "None" else mapping_dict[key_action[2]]
                       if obj11 == obj22:
                              return True
        return False
def match(list1, list2):
    if len(list2) == 0:
        return False
    index1 = 0
    index2 = 0
    
    while index1 < len(list1) and index2 < len(list2):
        if match_action(list1[index1], list2[index2], mapping_dict):
            index2 += 1
        index1 += 1
    
    return index2 == len(list2)


In [ ]:
eval_success_list = eval_success_list_gen(exp_name,round)
eval_success_list

In [ ]:
success_list = []

for eval_success in eval_success_list:
    task_id = eval_success['index']
    history = eval_success['history']
    if 1 <= int(task_id) <= 90:
        data_dir = "EMMOE-100/data/train"
    else:
        data_dir = "EMMOE-100/data/test"

    with open(os.path.join(data_dir, str(task_id), "keypath.json"), 'r') as file:
        keypaths = json.load(file)

    for keypath in keypaths:
        if match(history, keypath):
            success_list.append(task_id)
            break

print(len(success_list))
success_list

In [ ]:
for i in success_list:
    if i in ran:
        print(i)

In [ ]:
def count_dataset_path():
    path_count = 0
    for test_scene in os.listdir('EMMOE-100/data/test'):
        with open(f'EMMOE-100/data/test/{test_scene}/info.txt', 'r') as f:
            info = f.read()
            subtaskss = info.split('\n')
            subtasks = [  i for i in subtaskss if 'Subtask' in i]
            path_count += len(subtasks)
            # print(len(subtasks))
    for train_scene in os.listdir('EMMOE-100/data/train'):
        with open(f'EMMOE-100/data/train/{train_scene}/info.txt', 'r') as f:
            info = f.read()
            subtaskss = info.split('\n')
            subtasks = [  i for i in subtaskss if 'Subtask' in i]
            path_count += len(subtasks)
    return path_count

In [ ]:
def path_length_weighted(episode_id,eval_list):
    for aval in eval_list:
        if episode_id == aval['index']:
            real_path_count = aval['step_len']
    if episode_id > 90:
        temp = 'test'
    else:
        temp = 'train'
    with open(f'EMMOE-100/data/{temp}/{episode_id}/info.txt', 'r') as f:
        info = f.read()
        subtaskss = info.split('\n')
        subtasks = [  i for i in subtaskss if 'Subtask' in i]
        dataset_path_count = len(subtasks)
    return dataset_path_count/max(real_path_count,dataset_path_count), real_path_count, dataset_path_count

In [ ]:
a,b,c = path_length_weighted(episode_id,eval_list)
print(a,b,c)

In [ ]:
def eval_list_gen_ran(exp_name, round,ran):

    exp_path = os.path.join("/data1/tct_data/Video-LLaVA/exp", exp_name)

    eval_list = []

    for index in os.listdir(exp_path):
        conv_path = os.path.join(exp_path, index, f"{round}/conversation.json")
        if not os.path.exists(conv_path):
            continue
        if int(index) in ran:
            with open(conv_path, 'r') as file:
                conv = json.load(file)  
            exe_dict = {}
            exe_dict["index"] = int(index) 
            exe_dict["step_len"] = len(conv)

            history = conv[-1].split('\n')[3]
            exe_dict["history"] = parse_execution_history(history)
            eval_list.append(exe_dict) 

    eval_list = sorted(eval_list, key=lambda x: x["index"])
    print(len(eval_list))
    return eval_list
def plwsr(exp_name,ran):
    plw_total = 0
    task_total = 0
    for round in range(1,4):
        eval_list = eval_list_gen_ran(exp_name,round,ran)
        eval_success_list = eval_success_list_gen(exp_name,round)
        
        success_list = []
        for eval_success in eval_success_list:
            task_id = eval_success['index']
            history = eval_success['history']
            if 1 <= int(task_id) <= 90:
                data_dir = "/data1/tct_data/Video-LLaVA/HomieBot/data/train"
            else:
                data_dir = "/data1/tct_data/Video-LLaVA/HomieBot/data/test"

            with open(os.path.join(data_dir, str(task_id), "keypath.json"), 'r') as file:
                keypaths = json.load(file)

            for keypath in keypaths:
                if match(history, keypath):
                    success_list.append(task_id)
                    break

        print(len(success_list))
        for succ in success_list:
            if succ in ran:
                plw, _, _ = path_length_weighted(succ,eval_list)
                plw_total += plw
        task_total += len(eval_list)

    return plw_total/task_total
def sr(exp_name,ran):
    sr_total = 0
    task_total = 0
    for round in range(1,4):
        eval_list = eval_list_gen_ran(exp_name,round,ran)
        eval_success_list = eval_success_list_gen(exp_name,round)
        
        success_list = []
        for eval_success in eval_success_list:
            task_id = eval_success['index']
            history = eval_success['history']
            if 1 <= int(task_id) <= 90:
                data_dir = "/data1/tct_data/Video-LLaVA/HomieBot/data/train"
            else:
                data_dir = "/data1/tct_data/Video-LLaVA/HomieBot/data/test"

            with open(os.path.join(data_dir, str(task_id), "keypath.json"), 'r') as file:
                keypaths = json.load(file)

            for keypath in keypaths:
                if match(history, keypath):
                    success_list.append(task_id)
                    break

        print(len(success_list))
        for succ in success_list:
            if succ in ran:
                sr_total += 1
        task_total += len(eval_list)

    return sr_total/task_total

In [ ]:
sr(exp_name, ran)

In [ ]:
plwsr(exp_name,ran)

In [ ]:
real_path_count = 0
for i in eval_list:
    real_path_count += i["step_len"]
dataset_path_count = count_dataset_path()
print(real_path_count)
real_path_count/dataset_path_count

In [ ]:
20*100/count_dataset_path()